# Descargando datos en XML
### Rodrigo Iván González Valenzuela

In [16]:
import numpy as np
import pandas as pd
import os  # Para manejo de archivos y directorios
import urllib.request  # Para la descarga de los archivos 
import xml.etree.ElementTree as et # Para analizar el árbol generado XML
import re # Para el uso de expresiones regulares

poetas_xml_url = "https://github.com/mcd-unison/ing-caract/raw/main/ejemplos/\
    integracion/ejemplos/wikipedia-poetas.xml"
subdir= "ing-carac/xml-poetas/"
poetas_xml_archivo = subdir+"poetas.xml"
if not os.path.exists(poetas_xml_archivo):  # Comprueba si el archivo existe
    urllib.request.urlretrieve(poetas_xml_url, poetas_xml_archivo)  # Descarga los datos de la URL

Con la librería `xml.etree.ElementTree` se puede acceder a la estructura del archivo XML y obtener su elemento `root`. Posterior se buscan los elementos hijos que contengan los artículos de cada poeta para extraer la información que nos resulta útil: el nombre del poeta, el id del artículo y el texto biográfico.

In [17]:
tree = et.parse(poetas_xml_archivo)
root = tree.getroot()
print(root.tag)
df_col = ["id","nombre","texto"]
rows = []

# Extraer los datos de XML
for poeta in root.iter('{http://www.mediawiki.org/xml/export-0.10/}page'):
    id = poeta.find('{http://www.mediawiki.org/xml/export-0.10/}id').text
    nombre = poeta.find('{http://www.mediawiki.org/xml/export-0.10/}title').text
    docum = poeta.find('{http://www.mediawiki.org/xml/export-0.10/}revision/{http://www.mediawiki.org/xml/export-0.10/}text').text
    rows.append({"id": int(id), "nombre": nombre,"texto":docum})

poetas_df_raw = pd.DataFrame(rows, columns = df_col)

{http://www.mediawiki.org/xml/export-0.10/}mediawiki


El principal problema para obtener más datos útiles sobre los poetas es que el artículo se encuentra como un solo elemento `<text>`, por lo que es necesario limpiar el formato en que se obtuvo (eliminando y reemplazando caracteres) por medio de la librería `re` y las siguientes expresiones regulares:

In [18]:
# Prepara la columna "texto" con expresiones regulares 
poetas_text = [texto for texto in poetas_df_raw['texto'].values]

remplaza_por_espacios_re = re.compile('[\n/(){}\[\]\|@,;\.]')
simbolos_a_eliminar_re = re.compile('[^\d\w #+_\']')
def prepara_texto(texto):
    text = texto.lower()
    
    # Elimina simbolos
    text = re.sub(remplaza_por_espacios_re, ' ', text)
    text = re.sub(simbolos_a_eliminar_re, '', text)
    text= re.sub('\\s{2,}', ' ', text)
    
    return text
poetas_text_prep = [prepara_texto(texto) for texto in poetas_text]

In [19]:
#Busca las fechas de nacimiento de los poetas
poetas_df = poetas_df_raw.copy(deep = True)
poetas_df['texto'] = poetas_text_prep

fec_nac_list = []
for poeta in range(len(poetas_df.index)):
    fec_nac= re.search(r'[0-9]{1,2} de [A-Za-z]{1,10} de [0-9]{1,4}',poetas_df.loc[poeta,'texto'])
    if fec_nac != None :
        fec_nac_list.append(fec_nac[0])
    else:
        fec_nac_list.append("NaN")
poetas_df['fecha_nac']=fec_nac_list
print(poetas_df.head(20))

         id                      nombre  \
0   4949229   Julia Morilla de Campbell   
1   5105749                Luis Negreti   
2   4477192                  Poldy Bird   
3    423422              Ana María Shua   
4   4284479                León Benarós   
5   5436786  Alejandro González Gattone   
6   3148146              Silvia Schujer   
7   2428185              Laura Devetach   
8   1108758              Graciela Repún   
9   3222523        María Cristina Ramos   
10   190419                 Olga Orozco   
11   859323            Luisa Valenzuela   
12  5687611           Manuela Fingueret   
13  2605282            Javier Villafañe   
14   538911          Santiago Kovadloff   
15   505969            Luis Landriscina   
16  2703105                    Tom Lupo   
17  1905476        Alicia Dujovne Ortiz   
18  5708383               Rosa Wernicke   
19  6059383             Ataliva Herrera   

                                                texto                fecha_nac  
0   '''julia mo

Una vez que se tenga el dataframe con las características deseadas, lo podemos guardar en formato parquet.

In [26]:
poetas_df.to_parquet(path=subdir+"poetas.parquet",engine= 'pyarrow')